In [76]:
from sklearn.model_selection import train_test_split
import pandas as pd

In [77]:
df = pd.read_csv('TRNcod.csv', sep='\t')

In [78]:
df = df.drop(['INDEX', 'IND_BOM_1_2'], axis=1)

In [79]:
df.head(2)

,UF_1,UF_2,UF_3,UF_4,UF_5,UF_6,UF_7,IDADE,SEXO_1,NIVEL_RELACIONAMENTO_CREDITO01,...,CEP4_6,CEP4_7,CEP4_8,CEP4_9,CEP4_10,CEP4_11,CEP4_12,CEP4_13,CEP4_14,IND_BOM_1_1
0,1,1,1,0,0,0,0,0.135098,1,0.222222,...,0,0,0,1,1,0,1,1,1,0
1,1,0,1,0,0,1,0,0.273504,1,0.111111,...,0,0,1,0,1,1,0,0,0,1


#### **First step: splitting classes**

In [80]:
df_class0 = df[df.IND_BOM_1_1 == 0]
df_class1 = df[df.IND_BOM_1_1 == 1]

#### **Second step: sampling classes**

In [81]:
def split_classes(df):
    duplicated_mask = df.duplicated()

    df_duplicated = df[duplicated_mask]
    df_non_duplicated = df[~duplicated_mask]

    duplicated_percentage = len(df_duplicated) / len(df_non_duplicated)
    cut_bound = 0.5 - duplicated_percentage

    df_50_0, df_test = train_test_split(df_non_duplicated, test_size=cut_bound, random_state=42)

    df_25_0, df_25_1 = train_test_split(df_test, test_size=0.7, random_state=42, shuffle=False)
    
    return df_50_0, df_25_0, pd.concat([df_25_1, df_duplicated])

In [82]:
df_class0_50, df_class0_25_0, df_class0_25_1 = split_classes(df_class0)

In [83]:
df_class1_50, df_class1_25_0, df_class1_25_1 = split_classes(df_class1)

#### **Third step: joining samples && randomization**

In [84]:
df_train = pd.concat([df_class0_50, df_class1_50, df_class0_25_0, df_class1_25_0])
df_valid = pd.concat([df_class0_25_1, df_class1_25_1])

In [85]:
df_train = df_train.sample(frac=1)
df_valid = df_valid.sample(frac=1)

In [86]:
len(df_train) / len(df)

0.6500169580365677

In [87]:
len(df_valid) / len(df)

0.34998304196343233

In [88]:
df_train.to_csv('training.csv', index=False)
df_valid.to_csv('validation.csv', index=False)